importo i pacchetti necessari

In [111]:
import pandas as pd
import requests
import coordinates_calcolator as cc
import url_contructor as uc
import image_visualizator as iv
from db_manager import create_empty_csv, populate_csv

prendo i dati dal csv filtrato

In [112]:
mars_data = pd.read_csv('Mars_crater_db_filtered.csv')
zoom = 6
create_empty_csv('Mars_crater_db_images.csv', ['id', 'image'])

dato un valore di inizio e uno di fine da mars data faccio partire una request costruita con url_contructor con lat e lon da coordinates_calcolator

In [113]:
def get_data(start, end):
    data = []
    for i in range(start, end):
        lat = mars_data['lat'][i]
        print(lat)
        lon = mars_data['lon'][i]
        print(lon)
        id = mars_data['id'][i]
        print(id)
        lat, lon = cc.lat_lon_to_tile(lat, lon, zoom)
        print (lat, lon)
        url = uc.get_request_url(lat, lon, tileMatrix=zoom)
        print(url)
        response = requests.get(url)
        if response.status_code == 200:
            data.append([id,response.content])
        else:
            print('error')
    return data

In [114]:
data = get_data(0,10)


69.244
-27.24
01-000002
27 14
https://trek.nasa.gov/tiles/Mars/EQ/msss_atlas_simp_clon/1.0.0//default/default028mm/6/27/14.png
75.539
13.829
01-000008
34 10
https://trek.nasa.gov/tiles/Mars/EQ/msss_atlas_simp_clon/1.0.0//default/default028mm/6/34/10.png
77.707
-27.314
01-000010
27 9
https://trek.nasa.gov/tiles/Mars/EQ/msss_atlas_simp_clon/1.0.0//default/default028mm/6/27/9.png
84.687
1.589
01-000015
32 0
https://trek.nasa.gov/tiles/Mars/EQ/msss_atlas_simp_clon/1.0.0//default/default028mm/6/32/0.png
84.452
-18.503
01-000023
28 1
https://trek.nasa.gov/tiles/Mars/EQ/msss_atlas_simp_clon/1.0.0//default/default028mm/6/28/1.png
70.051
-7.939
01-000025
30 14
https://trek.nasa.gov/tiles/Mars/EQ/msss_atlas_simp_clon/1.0.0//default/default028mm/6/30/14.png
69.74
9.215
01-000033
33 14
https://trek.nasa.gov/tiles/Mars/EQ/msss_atlas_simp_clon/1.0.0//default/default028mm/6/33/14.png
68.373
6.224
01-000035
33 15
https://trek.nasa.gov/tiles/Mars/EQ/msss_atlas_simp_clon/1.0.0//default/default028mm/6/33


for i in range(len(data)):
    id = data[i][0]
    print(id)
    image = data[i][1]
    iv.image_visualizator_from_bin(image)


popolo il csv 'Mars_crater_db_images.csv' con i dati ottenuti

In [115]:
populate_csv(data)